In [6]:
#Importing packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dill

In [7]:
with open("df.pkl", "rb") as file:
    df = dill.load(file)
df.head()

,label,text,polarity,lemma_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",-0.044875,@switchfoot http://twitpic.com/2y1zl Awww bumm...
1,0,is upset that he can't update his Facebook by ...,-0.206880,upset update Facebook texte cry result Schoo...
2,0,@Kenichan I dived many times for the ball. Man...,0.246950,@Kenichan dive time ball manage save 50 rest...
3,0,my whole body feels itchy and like its on fire,-0.250000,body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",-0.123475,@nationwideclass behave mad


In [11]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import cred

credential = AzureKeyCredential(cred.cognitive_credential)
endpoint=cred.cognitive_endpoint

text_analytics_client = TextAnalyticsClient(endpoint, credential)

text_analytics_client

In [15]:
sample_df = df.head(5)

documents = sample_df["text"]

response = text_analytics_client.analyze_sentiment(documents.to_list(), language="en")
result = [doc for doc in response]

for doc in result:
    print(f"Overall sentiment: {doc.sentiment}")
    print(
        f"Scores: positive={doc.confidence_scores.positive}; "
        f"neutral={doc.confidence_scores.neutral}; "
        f"negative={doc.confidence_scores.negative}\n"
    )

Overall sentiment: negative
Scores: positive=0.0; neutral=0.0; negative=1.0

Overall sentiment: negative
Scores: positive=0.04; neutral=0.18; negative=0.78

Overall sentiment: positive
Scores: positive=0.98; neutral=0.01; negative=0.01

Overall sentiment: neutral
Scores: positive=0.05; neutral=0.6; negative=0.34

Overall sentiment: negative
Scores: positive=0.0; neutral=0.02; negative=0.98



In [16]:
sample_df.head()

,label,text,polarity,lemma_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",-0.044875,@switchfoot http://twitpic.com/2y1zl Awww bumm...
1,0,is upset that he can't update his Facebook by ...,-0.206880,upset update Facebook texte cry result Schoo...
2,0,@Kenichan I dived many times for the ball. Man...,0.246950,@Kenichan dive time ball manage save 50 rest...
3,0,my whole body feels itchy and like its on fire,-0.250000,body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",-0.123475,@nationwideclass behave mad


In [18]:
label = [1 if doc.confidence_scores.positive > doc.confidence_scores.negative else 0 for doc in result]

label

[0, 0, 1, 0, 0]

In [23]:
sample_df = df.head(500)

results = []
labels=[]

for i in range(1,51,1):
    k = 10*(i-1)
    l = 10*i
    subset = sample_df[k:l]
    
    documents = subset["text"]

    response = text_analytics_client.analyze_sentiment(documents.to_list(), language="en")
    result = [doc for doc in response]

    results.append(result)

    azure_label = [1 if doc.confidence_scores.positive > doc.confidence_scores.negative else 0 for doc in result]

    labels.append(azure_label)

In [38]:
flat_labels = list(np.concatenate(labels).flat)

sample_df["azure_label"] = flat_labels

sample_df.head()

,label,text,polarity,lemma_text,azure_label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",-0.044875,@switchfoot http://twitpic.com/2y1zl Awww bumm...,0
1,0,is upset that he can't update his Facebook by ...,-0.206880,upset update Facebook texte cry result Schoo...,0
2,0,@Kenichan I dived many times for the ball. Man...,0.246950,@Kenichan dive time ball manage save 50 rest...,1
3,0,my whole body feels itchy and like its on fire,-0.250000,body feel itchy like fire,0
4,0,"@nationwideclass no, it's not behaving at all....",-0.123475,@nationwideclass behave mad,0


In [41]:
from sklearn.metrics import accuracy_score

accuracy_score(sample_df["label"], sample_df["azure_label"])

0.718